In [ ]:
# install required packages 
#!pip install beautifulsoup4 lxml selenium webdriver-manager

In [1]:
# import necessary modules
import time
import requests
from csv import writer
from bs4 import BeautifulSoup
from lxml import etree as et
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# define job and location search keywords
job_search_keyword = ['Data+Scientist']#, 'Data+Analyst', 'Product+Analyst', 'BI+Analyst']
location_search_keyword = ['New+York']#, 'Los+Angeles', 'Chicago']

# define base and pagination URLs
base_url = 'https://www.indeed.com'
paginaton_url = "https://www.indeed.com/jobs?q={}&l={}&radius=35&start={}"

# check also Sweden and France 
# Just pick the 3 largest cities in each country 

In [7]:
# Function to get DOM from given URL
def get_dom(url):
    driver.get(url)
    time.sleep(3)  # Ensure page loads
    page_content = driver.page_source
    product_soup = BeautifulSoup(page_content, 'html.parser')
    dom = et.HTML(str(product_soup))
    return dom

# Function to extract job link
def get_job_link(job):
    try:
        return job.xpath('./descendant::h2/a/@href')[0]
    except Exception:
        return 'Not available'

# Function to extract job description
def get_job_desc(job_link):
    job_dom = get_dom(job_link)
    try:
        job_desc = job_dom.xpath('//*[@id="jobDescriptionText"]//text()')
        return " ".join(job_desc).strip() if job_desc else 'Not available'
    except Exception:
        return 'Not available'

def get_skills(job_link):
    job_dom = get_dom(job_link)
    try:
        # Use a refined XPath to get all relevant text within the profile insights
        skills = job_dom.xpath('//div[contains(@class, "profile-insights")]//text()')
        return " ".join(skills).strip() if skills else 'Not available'
    except Exception:
        return 'Not available'

# functions to extract job title
def get_job_title(job):
   try:
       job_title = job.xpath('./descendant::h2/a/span/text()')[0]
   except Exception as e:
       job_title = 'Not available'
   return job_title

# functions to extract the company name
def get_company_name(job):
   try:
       company_name = job.xpath('.//span[@data-testid="company-name"]/text()')[0]
       #company_name = job.xpath('./descendant::span[@class="companyName"]/text()')[0]
   except Exception as e:
       company_name = 'Not available'
   return company_name

# functions to extract the company location
def get_company_location(job):
   try:
       company_location = job.xpath('.//div[@data-testid="text-location"]/text()')[0]
       #company_location = job.xpath('./descendant::div[@class="companyLocation"]/text()')[0]
   except Exception as e:
       company_location = 'Not available'
   return company_location

# functions to extract salary information
'''
def get_salary(job):
   try:
       salary = job.xpath('./descendant::span[@class="estimated-salary"]/span/text()')
   except Exception as e:
       salary = 'Not available'
   if len(salary) == 0:
       try:
           salary = job.xpath('./descendant::div[@class="metadata salary-snippet-container"]/div/text()')[0]
       except Exception as e:
           salary = 'Not available'
   else:
       salary = salary[0]
   return salary
'''

def get_salary(job_link):
    job_dom = get_dom(job_link)
    try:
        # Use the provided XPath to get the salary text
        salary = job_dom.xpath('//*[@id="salaryInfoAndJobType"]//text()')
        return " ".join(salary).strip() if salary else 'Not available'
    except Exception:
        return 'Not available'
        
# functions to extract job type
def get_job_type(job):
   try:
       job_type = job.xpath('./descendant::div[@class="metadata"]/div/text()')[0]
   except Exception as e:
       job_type = 'Not available'
   return job_type

In [ ]:
# Initialize Chrome webdriver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# Open URL
driver.get("https://www.indeed.com/q-USA-jobs.html")

# Open a CSV file to write the job listings data
with open('indeed_jobs1.csv', 'w', newline='', encoding='utf-8') as f:
    theWriter = writer(f)
    heading = ['job_link', 'search_keyword', 'search_location', 'job_title', 'company_name', 'company_location', 'salary', 'skills', 'job_type', 'job_description']
    theWriter.writerow(heading)

    # Example job search keywords and locations
    job_search_keywords = ['Data Scientist']  # Replace with actual keywords
    location_search_keywords = ['New York']  # Replace with actual locations

    for job_keyword in job_search_keywords:
        for location_keyword in location_search_keywords:
            print(f"Searching for: {job_keyword} in {location_keyword}")
            all_jobs = []
            for page_no in range(2):  # Modify range to get more pages if needed
                url = f"https://www.indeed.com/jobs?q={job_keyword}&l={location_keyword}&start={page_no * 10}"
                page_dom = get_dom(url)
                jobs = page_dom.xpath('//div[@class="job_seen_beacon"]')
                all_jobs.extend(jobs)

            seen_links = set()
            for job in all_jobs:
                job_link = base_url + get_job_link(job)
                if job_link in seen_links:
                    continue  # Skip duplicates
                seen_links.add(job_link)

                job_title = get_job_title(job)
                company_name = get_company_name(job)
                company_location = get_company_location(job)
                salary = get_salary(job_link)
                job_type = get_job_type(job)
                job_desc = get_job_desc(job_link)  # Extract job description from the job link
                skills = get_skills(job_link)  # Extract skills

                record = [job_link, job_keyword, location_keyword, job_title, company_name, company_location, salary, skills, job_type, job_desc]

                print(salary)
                
                theWriter.writerow(record)

# Closing the web browser
driver.quit()

Searching for: Data Scientist in New York
Full-time
$84,981 - $97,728 a year  -   Full-time
$125,500 - $212,800 a year  -   Full-time
$137,600 - $294,000 a year  -   Full-time


In [ ]:
# for inspection and debugging 
'''

# initialize Chrome webdriver using ChromeDriverManager
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# open URL 
driver.get("https://www.indeed.com/q-USA-jobs.html?vjk=823cd7ee3c203ac3")
with open('indeed_jobs1_dom.txt', 'w', encoding='utf-8') as f:
    url = paginaton_url.format(job_search_keyword[0], location_search_keyword[0], page_no)
    time.sleep(3)
    page_dom = get_dom(url)  # Get the DOM of the page

    # Convert the DOM object to a string with pretty formatting
    dom_html = etree.tostring(page_dom, pretty_print=True, encoding='unicode')

    # Write the HTML string to the file
    f.write(dom_html)

# Save the full page DOM to a file for manual inspection
with open('indeed_full_page.html', 'w', encoding='utf-8') as f:
    time.sleep(4)
    page_source = driver.page_source
    f.write(page_source)
'''

In [ ]:
# ideas for visualization 
# number of listings per location
# average salary for each position 
# average salary by job AND location https://www.indeed.com/rc/clk?jk=9e205f9634a5bfdb&bb=Kaksi2QKb_5G01a0IYYWte3dB7Sd8vTC2PsDgK8Wcrl-vt-ov1bmllcNU4eq-ARapdqrzuLgp5TwUhCEMRJS0x0bX5YFJls_0xhvnMGs1P2Xvjh_c0znjkjJ1X6wishm&xkcb=SoBo67M37Ey6rjTbpx0LbzkdCdPP&fccid=2e5243142d98319d&vjs=3


# keyword extraction
# frequency differences for different job titles 
# tech stack keywords 